# note 필드를 json으로만 가져가려면?

In [1]:
import json

# 원본 JSON 파일 경로
input_file = 'audio_features.json'

# 추출된 데이터를 저장할 새로운 JSON 파일 경로
output_file = 'audio_features_note.json'

# JSON 파일 읽기
with open(input_file, 'r', encoding='utf-8') as f:
    data = json.load(f)

# 새로운 데이터 구조 생성
extracted_data = {
    "audio_features": []
}

# 각 항목에서 file_path와 note 추출
for feature in data.get("audio_features", []):
    extracted_entry = {
        "file_path": feature.get("file_path", ""),
        "note": feature.get("note", "")
    }
    extracted_data["audio_features"].append(extracted_entry)

# 새로운 JSON 파일로 저장
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(extracted_data, f, ensure_ascii=False, indent=4)

print(f"추출이 완료되었습니다. 새로운 파일: {output_file}")


추출이 완료되었습니다. 새로운 파일: audio_features_note.json


# pair data를 json으로 extracting 해야한다면?

In [2]:
import os
import json
import pandas as pd

# JSON 및 JSON 파일 경로 설정
json_file_path = "audio_features.json"
output_json_path = "audio_pair_notes.json"

def find_matching_normal_file(anomaly_path, normal_paths):
    """
    이상 파일에 대해 매칭되는 정상 파일을 찾는 함수.

    Args:
        anomaly_path (str): 이상 파일 경로.
        normal_paths (list): 정상 파일 경로 목록.

    Returns:
        str: 매칭된 정상 파일 경로 (없으면 None).
    """
    # 이상 파일 정보 추출
    anomaly_dir = os.path.dirname(anomaly_path)
    anomaly_filename = os.path.basename(anomaly_path)
    anomaly_number = anomaly_filename.split("_")[5]  # 파일 번호 추출
    anomaly_source_target = "source" if "source" in anomaly_filename else "target"  # source/target 확인

    # 같은 디렉토리 내에서 정상 파일 매칭
    for normal_path in normal_paths:
        normal_dir = os.path.dirname(normal_path)
        normal_filename = os.path.basename(normal_path)
        normal_number = normal_filename.split("_")[5]  # 파일 번호 추출
        normal_source_target = "source" if "source" in normal_filename else "target"  # source/target 확인

        # 같은 디렉토리, 같은 파일 번호, 같은 source/target 조건
        if (anomaly_dir == normal_dir and
            anomaly_number == normal_number and
            anomaly_source_target == normal_source_target):
            return normal_path

    print(f"No matching normal file found for: {anomaly_path}")
    return None


def create_audio_pair_json(json_file_path, output_json_path):
    """
    JSON 데이터를 읽어 이상/정상 페어를 JSON으로 저장.

    Args:
        json_file_path (str): JSON 파일 경로.
        output_json_path (str): 저장할 JSON 파일 경로.
    """
    # JSON 데이터 로드
    with open(json_file_path, 'r', encoding='utf-8') as f:
        audio_features_data = json.load(f)

    # normal, anomaly 파일 분류
    normal_files = []
    anomaly_files = []
    notes = {}

    for feature in audio_features_data.get("audio_features", []):
        file_path = feature.get("file_path", "")
        note = feature.get("note", "")
        notes[file_path] = note

        # 경로에 따라 anomaly/normal 분류
        if "anomaly" in os.path.basename(file_path).lower():
            anomaly_files.append(file_path)
        else:
            normal_files.append(file_path)

    # 페어 찾기
    pair_data = []
    for anomaly_file in anomaly_files:
        matching_normal_file = find_matching_normal_file(anomaly_file, normal_files)
        if matching_normal_file:
            pair_data.append({
                "normal_file_path": matching_normal_file,
                "anomaly_file_path": anomaly_file,
                "normal_note": notes.get(matching_normal_file, ""),
                "anomaly_note": notes.get(anomaly_file, "")
            })

    # 추가: 머신 타입 및 파일 번호 추출 (선택 사항)
    for pair in pair_data:
        # 머신 타입 추출 예: '../unziped/dev/ToyCar/test/section_00_source_test_normal_0027_car_C2_spd_34V_mic_1.wav'
        # 머신 타입은 4번째 요소 ('ToyCar')
        try:
            pair["machine_type"] = pair["normal_file_path"].split("/")[3]
        except IndexError:
            pair["machine_type"] = "Unknown"

        # 파일 번호 추출 예: '0027'
        try:
            pair["file_number"] = os.path.basename(pair["normal_file_path"]).split("_")[4]
        except IndexError:
            pair["file_number"] = "Unknown"

    # JSON 구조 정의
    output_data = {
        "audio_pairs": pair_data
    }

    # JSON 저장
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(output_data, f, ensure_ascii=False, indent=4)
    
    print(f"페어 데이터가 JSON으로 저장되었습니다: {output_json_path}")
    print(f"총 {len(pair_data)}개의 페어가 생성되었습니다.")

    # 추가: 정렬 (선택 사항)
    # 머신 타입과 파일 번호에 따라 정렬하려면 다음과 같이 할 수 있습니다.
    # output_data["audio_pairs"] = sorted(pair_data, key=lambda x: (x["machine_type"], x["file_number"]))
    # 위의 정렬을 원한다면, sorted 함수의 주석을 해제하고 다시 저장하면 됩니다.

# 실행
if __name__ == "__main__":
    create_audio_pair_json(json_file_path, output_json_path)


페어 데이터가 JSON으로 저장되었습니다: audio_pair_notes.json
총 700개의 페어가 생성되었습니다.


# handwrite csv 파일이 필요한 경우 json에서 따로 추출

In [ ]:
import json
import pandas as pd

# JSON 파일 경로
json_file_path = "audio_features.json"  # 실제 JSON 파일 경로로 변경
output_csv_path = "audio_features_notes.csv"  # 출력 CSV 파일 경로

# JSON 파일 읽기
with open(json_file_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

# JSON 데이터에서 file_path와 note 필드만 추출
audio_features = data.get("audio_features", [])
df = pd.DataFrame(audio_features)[["file_path", "note"]]

# CSV 파일로 저장
df.to_csv(output_csv_path, index=False)

print(f"CSV 파일이 생성되었습니다: {output_csv_path}")


CSV 파일이 생성되었습니다: audio_features_notes.csv


# pair data with csv

In [25]:
import os
import json
import pandas as pd

# JSON 및 CSV 파일 경로 설정
json_file_path = "audio_features.json"
output_csv_path = "audio_pair_notes.csv"

def find_matching_normal_file(anomaly_path, normal_paths):
    """
    이상 파일에 대해 매칭되는 정상 파일을 찾는 함수.

    Args:
        anomaly_path (str): 이상 파일 경로.
        normal_paths (list): 정상 파일 경로 목록.

    Returns:
        str: 매칭된 정상 파일 경로 (없으면 None).
    """
    # 이상 파일 정보 추출
    anomaly_dir = os.path.dirname(anomaly_path)
    anomaly_filename = os.path.basename(anomaly_path)
    anomaly_number = anomaly_filename.split("_")[5]  # 파일 번호 추출
    anomaly_source_target = "source" if "source" in anomaly_filename else "target"  # source/target 확인

    # 같은 디렉토리 내에서 정상 파일 매칭
    for normal_path in normal_paths:
        normal_dir = os.path.dirname(normal_path)
        normal_filename = os.path.basename(normal_path)
        normal_number = normal_filename.split("_")[5]  # 파일 번호 추출
        normal_source_target = "source" if "source" in normal_filename else "target"  # source/target 확인

        # 같은 디렉토리, 같은 파일 번호, 같은 source/target 조건
        if (anomaly_dir == normal_dir and
            anomaly_number == normal_number and
            anomaly_source_target == normal_source_target):
            return normal_path

    print(f"No matching normal file found for: {anomaly_path}")
    return None

def create_audio_pair_csv(json_file_path, output_csv_path):
    """
    JSON 데이터를 읽어 이상/정상 페어를 CSV로 저장.

    Args:
        json_file_path (str): JSON 파일 경로.
        output_csv_path (str): 저장할 CSV 파일 경로.
    """
    # JSON 데이터 로드
    with open(json_file_path, 'r', encoding='utf-8') as f:
        audio_features_data = json.load(f)

    # normal, anomaly 파일 분류
    normal_files = []
    anomaly_files = []
    notes = {}

    for feature in audio_features_data["audio_features"]:
        file_path = feature["file_path"]
        note = feature["note"]
        notes[file_path] = note

        # 경로에 따라 anomaly/normal 분류
        if "anomaly" in os.path.basename(file_path).lower():
            anomaly_files.append(file_path)
        else:
            normal_files.append(file_path)

    # 페어 찾기
    pair_data = []
    for anomaly_file in anomaly_files:
        matching_normal_file = find_matching_normal_file(anomaly_file, normal_files)
        if matching_normal_file:
            pair_data.append({
                "normal_file_path": matching_normal_file,
                "anomaly_file_path": anomaly_file,
                "normal_note": notes.get(matching_normal_file, ""),
                "anomaly_note": notes.get(anomaly_file, "")
            })

    # 데이터프레임 생성 및 정렬
    pair_df = pd.DataFrame(pair_data)
    pair_df["machine_type"] = pair_df["normal_file_path"].apply(lambda x: x.split("/")[3])  # machine_type 추출
    pair_df["file_number"] = pair_df["normal_file_path"].apply(lambda x: os.path.basename(x).split("_")[5])  # 번호 추출
    pair_df.sort_values(by=["machine_type", "file_number"], inplace=True)
    pair_df.drop(columns=["machine_type", "file_number"], inplace=True)

    # CSV 저장
    pair_df.to_csv(output_csv_path, index=False, encoding="utf-8")
    print(f"페어 데이터가 CSV로 저장되었습니다: {output_csv_path}")

# 실행
if __name__ == "__main__":
    create_audio_pair_csv(json_file_path, output_csv_path)


페어 데이터가 CSV로 저장되었습니다: audio_pair_notes.csv
